In [7]:
import os
from langchain import PromptTemplate
import openai
import pinecone
from fastapi.middleware.cors import CORSMiddleware
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.conversational_retrieval.prompts import QA_PROMPT
from dotenv import load_dotenv, find_dotenv
from langchain.llms import AzureOpenAI
from langchain.memory import ConversationBufferWindowMemory
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA

#check .env
_ = load_dotenv(find_dotenv())

# Open AI Chat Completion LLM Model from Langchain, with Streaming enabled
openai_llm = AzureOpenAI(
    deployment_name="gpt-tech-snacks",
    model_name="text-embedding-ada-002",
    temperature=0
)

embeddings = OpenAIEmbeddings(openai_api_key=os.environ['OPENAI_API_KEY'],
                              #this makes it work with azure
                              deployment="embedding-tech-snacks",
                              model="text-embedding-ada-002",
                              openai_api_base=os.environ['OPENAI_API_BASE'],
                              openai_api_type=os.environ['OPENAI_API_TYPE']
                              )#1 is azure limitation
TEXT_FIELD = "text"

pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_REGION']  # find next to API key in console
)

index = pinecone.Index(os.environ['PINECONE_INDEX'])
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [8]:
namespace="helicopter"
#namespace="helicopter-big"
#namespace="sphinx"

In [9]:
vectorstore = Pinecone(
    index, embeddings.embed_query, TEXT_FIELD,
    namespace=namespace
)
qa = RetrievalQA.from_chain_type(
    llm=openai_llm,
    chain_type="stuff",    
    retriever=vectorstore.as_retriever(),
    memory=memory
)
qa.verbose=True

In [11]:
response = qa.run("what to check before take-off?")
print(response)

PineconeProtocolError: Failed to connect; did you specify the correct index name?

In [6]:
find_dotenv()

'c:\\Users\\csms\\github\\GFT.TechSnacks.LLMs\\.env'